# Room Detection AI

Creating a room detection AI from scratch. 

Aim: to create a GANs that can identify which room in the house the photo is of, and whether or not the photo is external or internal. 

The classes to explore are:

- Bathroom
- Bedroom
- Kitchen
- Conservatory
- Dinning Room
- Entrance Hall
- Living Room
- Landing
- Pool
- Pool House
- Study
- Utility Room
- Drawing Room
- Library 
- Cinema
- Basement
- Attic
- Courtyard 
- Pantry
- Shed
- Balcony 
- Loft Conversion
- Barn Converstion
- Garage
- Games room 
- Wet room 
- Wine / Cellar
- Staircase

## Data Collection

Potential sources for data scrapping:

- airbnb
- pinterest
- interior design websites


## Practice Scrape

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.istockphoto.com/photos/interior-design?mediatype=photography&phrase=interior%20design&sort=best'

In [3]:
# set response variable equal to the get function of the requests library
response = requests.get(url, headers={"Accept-Language":"en-US"}) # make sure the language of the retrieved scraping is english
# set soup variable
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
# find all the img 
soup.find_all("img", class_="gallery-asset__thumb gallery-mosaic-asset__thumb")

[<img class="gallery-asset__thumb gallery-mosaic-asset__thumb" src="https://media.istockphoto.com/photos/modern-living-room-in-the-evening-picture-id1203029609?k=6&amp;m=1203029609&amp;s=612x612&amp;w=0&amp;h=XXA-Tcq55KyJqFKP5MWxvyojbwsHHSxihJpqNS23FAI="/>,
 <img class="gallery-asset__thumb gallery-mosaic-asset__thumb" src="https://media.istockphoto.com/photos/stylish-composition-of-bedroom-interior-with-wooden-bed-design-shelf-picture-id1197277473?k=6&amp;m=1197277473&amp;s=612x612&amp;w=0&amp;h=1_SwRgI4RmYqB9zHFqdYKXdMBrjqQz6QqRqSgu5Z-5I="/>,
 <img class="gallery-asset__thumb gallery-mosaic-asset__thumb" src="https://media.istockphoto.com/photos/full-furnished-living-room-picture-id1149521187?k=6&amp;m=1149521187&amp;s=612x612&amp;w=0&amp;h=kUNSsZ3z1c9Iflsw6YD8Pofrs0H2oR-uOe58BjSpJhY="/>,
 <img class="gallery-asset__thumb gallery-mosaic-asset__thumb" src="https://media.istockphoto.com/photos/blue-prints-color-swatch-pencil-colors-sketches-plans-and-documents-picture-id1181691218?k=6&

In [5]:
import sys
!{sys.executable} -m pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [17]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [29]:
from PIL import Image
import requests
import os
from selenium import webdriver
import time 
import io
import hashlib


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    for elem in res:
        persist_image(target_folder,elem)


In [30]:
search_term = 'living room'

search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDxp9a9oymmY5xFEdHychsI-NPEPHJRNQyWQ&usqp=CAU - as ./images/living_room/3ed891d2e5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT1h9wtcyb4jLwTP3F7Z4gKEqu2nUoghw5fdg&usqp=CAU - as ./images/living_room/102acf569b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQywjiUdZqN7wEB5zgg29CVHXGdTHX5_mYddQ&usqp=CAU - as ./images/living_room/20be51b4d1.jpg
SUCCESS - saved https://www.telegraph.co.uk/content/dam/lifestyle/spark/living-room-inspiration/JL-small-living-rooms-main.jpg - as ./images/living_room/16ce2de506.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtraKEoPUBVCkcWPb9hZnSVbcokwBWqLN3Og&usqp=CAU - as ./images/living_room/18b7c54215.jpg
SUCCESS - saved https://www.storyhomes.co.uk/wp-content/uploads/2018/10/Cosify-1.png - as ./images/living_room/02e14306

SUCCESS - saved https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/living-room-6-1573660110.jpg?crop=1.00xw:0.781xh;0.00160xw,0.207xh&resize=1200:* - as ./images/living_room/34030fdc6f.jpg
SUCCESS - saved https://www.thespruce.com/thmb/WEst9BHwmGGE6AtmKrE5ck4m3t4=/3232x2424/smart/filters:no_upscale()/how-to-arrange-living-room-furniture-1976578-hero-c99074dcad854b669b91652046a39203.jpg - as ./images/living_room/224dad0557.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRzUgmwmBbB_eZIEKdwfVgmDRkXGLLi5QhtBg&usqp=CAU - as ./images/living_room/cf99eb80b3.jpg
SUCCESS - saved https://i.pinimg.com/originals/f2/32/11/f23211092978f90875f2ccfe7b2419c5.png - as ./images/living_room/0cc93325d4.jpg
SUCCESS - saved https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/white-living-room-sunny-1582128304.jpg - as ./images/living_room/8ddefa3795.jpg
SUCCESS - saved https://cdn.vox-cdn.com/thumbor/mzq4g3lgBnqzBY1-v0R2bt-dQSk=/0x0:4000x2667/1200x800/filters:foca

SUCCESS - saved https://cdn.shopify.com/s/files/1/0004/4630/0222/files/How_to_Decorate_a_Large_Living_Room___Ideas_Tips___LuxDeco.com.jpg?v=1597237353 - as ./images/living_room/e3def99ffd.jpg
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F23%2F2009%2F06%2F17%2Fdabito-living-room.png - as ./images/living_room/7b3add522d.jpg
